# Ruled Based Model
### Hey Together,   
Welcome to the first notebook for the ruled based model of commfy

   
## Part 1 import libarys

In [103]:
import random
from random import randint


## Part 2 gether Input
there is the option to give input yourself, to auto generate it once or to use the function input_generator() whenever you need it.

In [104]:
## auto generated as a function to use later:
def input_generator():
    weather=randint(1, 4)
    temperature=round(random.uniform(-10,35),2)
    if weather==1: weather='sunny'
    if weather==2: weather='rainy'
    if weather==3: 
        weather='snowy'
        temperature=min(temperature,4.)
    if weather==4: weather='windy'   
    heaviness_of_trip=randint(1,3)
    if heaviness_of_trip==1: heaviness_of_trip="easy"
    if heaviness_of_trip==2: heaviness_of_trip="normal"
    if heaviness_of_trip==3: heaviness_of_trip="hard"
    heat_preference=randint(-5,5)
    sex=randint(1,3)
    if sex==1: sex="m"
    if sex==2: sex="f"
    if sex==3: sex="d"
    return weather,temperature,heaviness_of_trip,heat_preference,sex
def print_input(list):
    print('the input for...\nweather is: ',list[0],'\ntemperature is: ',list[1],'\nheaviness of the trip is: ',list[2],'\nHeat_preference is: ',list[3],'\nsex is: ',list[4])

In [18]:
# ask for weather, temperature, duration and the sex
weather=input('please describe the weather as sunny; rainy; snowy or windy :  ')
temperature=input('please insert the temperature :  ')
temperature=float(temperature)
heaviness_of_trip=input('please insert the Heaviness of the trip as : easy, normal or hard ')
heat_preference=input('please insert your heat preference on a scal from -5 to 5: ')
sex=input('please tell me you sex as f, m or d  ')
print('your input for...\nweather is: ',weather,'\ntemperature is: ',temperature,'\nheaviness of the trip is: ',heaviness_of_trip,'\nheat preference is: ',heat_preference,'\nsex is: ',sex)
input_selfgenerated=(weather,temperature,heaviness_of_trip,heat_preference,sex)

please describe the weather as sunny; rainy; snowy or windy :  sunny
please insert the temperature :  20
please insert the Heaviness of the trip as : easy, normal or hard easy
please insert your heat preference on a scal from -5 to 5: -4
please tell me you sex as f, m or d  f
your input for...
weather is:  sunny 
temperature is:  20.0 
heaviness of the trip is:  easy 
heat preference is:  -4 
sex is:  f


In [19]:
print_input(input_selfgenerated)

the input for...
weather is:  sunny 
temperature is:  20.0 
heaviness of the trip is:  easy 
Heat_preference is:  -4 
sex is:  f


In [105]:
#test input generator
z=input_generator()
print(z, ' -->')
print_input(z)

('rainy', 14.24, 'easy', 3, 'd')  -->
the input for...
weather is:  rainy 
temperature is:  14.24 
heaviness of the trip is:  easy 
Heat_preference is:  3 
sex is:  d


<hr style="border:2px solid gray"> </hr>   
   
## Part 3 define the model
output of this model will be the prediciton for one single trip

In [150]:
## sunglasses decisiontree
def sunglasses(list):
    if list[0]=='sunny':
      if list[1]>20: return 1
    return 0
## neck decision tree
def neck(list):
    red=0.
    if list[4]=='f':
       red=2.
    if (list[1]+list[3]-red)>10:
       if ((list[1]+list[3]-red)<20) & (list[0]=='rainy'):
           return 2
       return 1
    if (list[1]+list[3]-red)>0:
       return 2
    if list[0]=='sunny':
       return 2
    return 3
##summing up the decision trees into 1 list
def get_clothing_output_rbm(list):
    a={}
    a["sunglasses"]=sunglasses(list)
    a["neck"]=neck(list)
    return a

In [215]:
#test the model
for i in range(20):
  a=input_generator()
  print(i+1,'  generated input: ',a, '  -->   ',get_clothing_output_rbm(a))


1   generated input:  ('sunny', -8.5, 'hard', 2, 'd')   -->    {'sunglasses': 0, 'neck': 2}
2   generated input:  ('snowy', 4.0, 'easy', 3, 'm')   -->    {'sunglasses': 0, 'neck': 2}
3   generated input:  ('sunny', 15.04, 'easy', 3, 'd')   -->    {'sunglasses': 0, 'neck': 1}
4   generated input:  ('snowy', -6.34, 'normal', -3, 'f')   -->    {'sunglasses': 0, 'neck': 3}
5   generated input:  ('windy', -2.64, 'normal', 5, 'f')   -->    {'sunglasses': 0, 'neck': 2}
6   generated input:  ('sunny', 12.73, 'normal', 4, 'f')   -->    {'sunglasses': 0, 'neck': 1}
7   generated input:  ('sunny', 34.63, 'hard', -1, 'm')   -->    {'sunglasses': 1, 'neck': 1}
8   generated input:  ('windy', 11.41, 'normal', 4, 'd')   -->    {'sunglasses': 0, 'neck': 1}
9   generated input:  ('snowy', -1.83, 'easy', 2, 'f')   -->    {'sunglasses': 0, 'neck': 3}
10   generated input:  ('rainy', 30.82, 'hard', 0, 'f')   -->    {'sunglasses': 0, 'neck': 1}
11   generated input:  ('windy', 2.73, 'normal', 2, 'f')   -->

<hr style="border:2px solid gray"> </hr>  

## Part 5: output concatenating
give out 2 lists (one for wearing now and one for the backpack)
translate the string into a Json for the Flask API

In [216]:
#autogenerate list of multiple input lists
def generate_multilist(a):
    multilist=[]
    for i in range(a): #a Trips today
       multilist.append(input_generator())
    return multilist

In [217]:
# get two lists for to wear now and to bring in the backpack
def get_clothing_multilist(multilist):
    ##wear_now is the first clothing set from the first input     
    # todo to reevaluate the list, if the big stuff like shoes or jackets just the warmest are choosen
    wear_now=get_clothing_output_rbm(multilist[0])

    backpack={}
    for key in get_clothing_output_rbm(multilist[0]).keys():
        for i in range(len(multilist)):
            if (get_clothing_output_rbm(multilist[i])[key]!=wear_now[key])&(key not in backpack.keys())&(get_clothing_output_rbm(multilist[i])[key]!=0):
                backpack[key]= get_clothing_output_rbm(multilist[i])[key]
            ##it should be just one piece per categorie in the backpack (if questioned, then the warmest(highest number))
            if (get_clothing_output_rbm(multilist[i])[key]!=wear_now[key])&(key in backpack.keys()):
                if get_clothing_output_rbm(multilist[i])[key]>backpack[key]:
                    backpack[key]=get_clothing_output_rbm(multilist[i])[key]
    return wear_now, backpack        
    

In [213]:
a= generate_multilist(5)
print(type(a),a)
get_clothing_multilist(a)

<class 'list'> [('snowy', 4.0, 'easy', 5, 'f'), ('rainy', 31.61, 'easy', -2, 'f'), ('rainy', 34.28, 'hard', -3, 'm'), ('windy', -5.74, 'easy', 2, 'm'), ('windy', 11.44, 'easy', -2, 'f')]


({'neck': 2, 'sunglasses': 0}, {'neck': 3})